In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd drive/MyDrive/Colab-workspace/

/content/drive/MyDrive/Colab-workspace


In [ ]:
# Setup up YOLOv5 Environment
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -U -r yolov5/requirements.txt  # install dependencies

Cloning into 'yolov5'...
remote: Enumerating objects: 9914, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9914 (delta 0), reused 0 (delta 0), pack-reused 9906
Receiving objects: 100% (9914/9914), 10.31 MiB | 31.13 MiB/s, done.
Resolving deltas: 100% (6865/6865), done.
     |████████████████████████████████| 10.3 MB 5.2 MB/s 
     |████████████████████████████████| 15.7 MB 35.6 MB/s 
     |████████████████████████████████| 60.3 MB 64 kB/s 
     |████████████████████████████████| 3.1 MB 49.7 MB/s 
     |████████████████████████████████| 596 kB 46.3 MB/s 
     |████████████████████████████████| 62 kB 735 kB/s 
     |████████████████████████████████| 38.2 MB 30 kB/s 
     |████████████████████████████████| 11.3 MB 48.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: Pillow
    Found 

In [ ]:
%cd yolov5

/content/yolov5


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

Tesla P100-PCIE-16GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [ ]:
import json

from tqdm import tqdm
from utils.general import download, Path

dir = Path('../datasets/Argoverse')  # dataset root dir
urls = ['https://argoverse-hd.s3.us-east-2.amazonaws.com/Argoverse-HD-Full.zip']
download(urls, dir=dir, delete=False)


def argoverse2yolo(set):
    labels = {}
    a = json.load(open(set, "rb"))
    for annot in tqdm(a['annotations'], desc=f"Converting {set} to YOLOv5 format..."):
        img_id = annot['image_id']
        img_name = a['images'][img_id]['name']
        img_label_name = img_name[:-3] + "txt"

        cls = annot['category_id']  # instance class id
        x_center, y_center, width, height = annot['bbox']
        x_center = (x_center + width / 2) / 1920.0  # offset and scale
        y_center = (y_center + height / 2) / 1200.0  # offset and scale
        width /= 1920.0  # scale
        height /= 1200.0  # scale

        img_dir = set.parents[2] / 'Argoverse-1.1' / 'labels' / a['seq_dirs'][a['images'][annot['image_id']]['sid']]
        if not img_dir.exists():
            img_dir.mkdir(parents=True, exist_ok=True)

        k = str(img_dir / img_label_name)
        if k not in labels:
            labels[k] = []
        labels[k].append(f"{cls} {x_center} {y_center} {width} {height}\n")

    for k in labels:
        with open(k, "w") as f:
            f.writelines(labels[k])

  0%|          | 0.00/29.2G [00:00<?, ?B/s]

Unzipping ../datasets/Argoverse/Argoverse-HD-Full.zip...


In [ ]:
annotations_dir = 'Argoverse-HD/annotations/'
(dir / 'Argoverse-1.1' / 'tracking').rename(dir / 'Argoverse-1.1' / 'images')  # rename 'tracking' to 'images'
for d in "train.json", "val.json":
    argoverse2yolo(dir / annotations_dir / d)

Converting ../datasets/Argoverse/Argoverse-HD/annotations/train.json to YOLOv5 format...: 100%|██████████| 771774/771774 [00:35<00:00, 21776.87it/s]
Converting ../datasets/Argoverse/Argoverse-HD/annotations/val.json to YOLOv5 format...: 100%|██████████| 253941/253941 [00:11<00:00, 21242.94it/s]


In [ ]:
!python train.py --data Argoverse.yaml --epochs 10 --batch-size 64

train: weights=yolov5s.pt, cfg=, data=Argoverse.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=10, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-89-g540ef0d torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, sc

In [ ]:
ls

sample_data/


In [ ]:
!ls

sample_data
